In [ ]:
#default_exp model_training_03

# Model Training
> Improved and refactored FCN modules as well as Parameters, Optimizers and a Training Loop

In [ ]:
#export
from ModernArchitecturesFromScratch.basic_operations_01 import *
from ModernArchitecturesFromScratch.fully_connected_network_02 import *
from nbdev.showdoc import *

In [ ]:
#hide
n_in = 784
n_out = 10

In [ ]:
#hide
xt, yt, xv, yv = get_mnist()
layers = [Linear(n_in,50, True), ReLU(), Linear(50,n_out, False)]
model = Model(layers)

## Better Cross Entropy

In [ ]:
#hide
def log_softmax(inp): 
    "Return the log softmax of input"
    return (inp.exp() / inp.exp().sum(-1, keepdim=True)).log()

In [ ]:
#export
def logsumexp(inp):
    "Helper function to compute log_softmax"
    a = inp.max(dim=1).values
    return a + ((inp-a[:,None]).exp().sum(-1)).log()

def log_softmax_improved(inp):
    "Improved `log_softmax` to take advantage of exponential properties"
    return inp - logsumexp(inp).unsqueeze(-1)

```python
def logsumexp(inp):
    a = inp.max(dim=1).values
    return a + ((inp-a[:,None]).exp().sum(-1)).log()

def log_softmax_improved(inp):
    return inp - logsumexp(inp).unsqueeze(-1)
```

In [ ]:
#export
def nll(inp, targ): 
    "Computes near log likelihood"
    return -(inp[range(targ.shape[0]), targ.long()].mean())

In [ ]:
#export
def cross_entropy(inp, targ): 
    "Uses `log_softmax_improved` and `nll` to compute cross entropy loss"
    return nll((log_softmax_improved(inp)), targ)

## Initial Train Loop

In [ ]:
#export
def accuracy(preds, targ): 
    "Compute accuracy of `preds` with respect to `targ`"
    return (torch.argmax(preds, dim=1)==targ).float().mean()

In [ ]:
#hide
bs = 64
lr = 0.5

In [ ]:
#export
def fit(epochs, model, loss_func, train, valid):
    "Basic function to fit `model` for given number of `epochs` with `loss_func` and `train` and `valid`. Prints out accuracy after every epoch. This will become heavily refactored"
    for epoch in range(epochs):
        for batch in range(math.ceil(len(train)//bs)):
            start = batch*bs
            end = batch*bs + bs
            train_batch = train[start:end]
            valid_batch = valid[start:end]
            
            loss = loss_func(model(train_batch), valid_batch)
            loss_func.backward(loss, )
            model.backward()
            
            with torch.no_grad():
                for l in model.layers:
                    if hasattr(l, 'w'):
                        l.w -= l.w.g * lr
                        l.b   -= l.b.g   * lr
                        l.w.g = 0
                        l.b.g = 0
        print(f'Epoch {epoch+1}, Accuracy: {accuracy(model(xt), yt)}')

```python
def fit(epochs, model, loss_func, train, valid):
    for epoch in range(epochs):
        for batch in range(math.ceil(len(train)//bs)):
            start = batch*bs
            end = batch*bs + bs
            train_batch = train[start:end]
            valid_batch = valid[start:end]
            
            loss = loss_func(model(train_batch), valid_batch)
            loss_func.backward(loss, )
            model.backward()
            
            with torch.no_grad():
                for l in model.layers:
                    if hasattr(l, 'w'):
                        l.w -= l.w.g * lr
                        l.b   -= l.b.g   * lr
                        l.w.g = 0
                        l.b.g = 0
        print(f'Epoch {epoch+1}, Accuracy: {accuracy(model(xt), yt)}')
        ```

## Parameters

In [ ]:
#export
class Parameter():
    "Defines a base class for all parameters that need to be learned by the model"
    def __init__(self, data=None, requires_grad=True):
        self.d = torch.Tensor() if data is None else data
        self.requires_grad = requires_grad
        self.grad = 0.
    
    def step(self, lr): 
        self.d -= lr * self.grad
    
    def zero_grad(self): self.grad = 0.
    
    def update(self, amount): self.grad = amount
        
    def __get__(self, instance, owner): return self.d
        
    def __repr__(self): 
        return f'Parameter: {self.d.shape}, Requires grad: {self.requires_grad}'

In [ ]:
l_data = torch.zeros([4,6])
s_data = torch.randn([4,2])
param_l = Parameter(l_data)
param_s = Parameter(s_data)

In [ ]:
param_l

Parameter: torch.Size([4, 6]), Requires grad: True

In [ ]:
param_s

Parameter: torch.Size([4, 2]), Requires grad: True

## Sequential Model 

In [ ]:
#export
class SequentialModel():
    "Model for executing forward and backward passes on a given list of `layers`"
    def __init__(self, *args):
        self.layers = list(args)
        self.training = True
    
    def __repr__(self):
        "Prints out all modules of model"
        res = ["(Layer" + str(i+1) + "): " + str(m) for i,m in enumerate(self.layers)]
        return "\n".join(res)
    
    def __call__(self, x):
        "Execute forward pass on `x` throuh `self.layers`"
        for l in self.layers: x = l(x)
        return x
    
    def backward(self):
        "Execute backward pass on `x` throuh `self.layers`"
        for l in reversed(self.layers): l.backward()
    
    def parameters(self):
        "Get iterator over all parameters in layers of `self.layers`"
        for l in self.layers:
            for p in l.parameters(): yield p

## FCN Layers Redo

In [ ]:
#export
class Module():
    "Defines a base class for all layers of the network. Allows for easy implementation of forward and backward passes as well as updating learnable parameters"
    def __init__(self):
        self._params = {}
        
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out

    def forward(self): raise Exception("Not Implemented")

    def backward(self): self.bwd(self.out, *self.args)  
        
    def __setattr__(self,k,v):
        if isinstance(v, Parameter): self._params[k] = v  
        super().__setattr__(k,v)
    
    def parameters(self):
        for p in self._params.values(): yield p

In [ ]:
#export

class CrossEntropy(Module):
    def forward(self, inp, targ):
        return cross_entropy(inp, targ)
    
    def bwd(self, loss, inp, targ):
        inp_s = softmax(inp)
        inp_s[range(targ.shape[0]), targ.long()] -= 1
        inp.g = inp_s / targ.shape[0]  

In [ ]:
#export

class Linear(Module):
    def __init__(self, in_d, out_d, relu_after, req_grad=True): 
        super().__init__()
        self.w = Parameter(get_weight(in_d, out_d, relu_after), req_grad)
        self.b = Parameter(torch.zeros(out_d), req_grad)

    def forward(self, xb): return xb @ self.w.d + self.b.d

    def bwd(self, out, inp):
        inp.g = out.g @ self.w.d.t()
        self.w.update(inp.t() @ out.g)
        self.b.update(out.g.sum(0))
    
    def __repr__(self): return f'Linear({self.w.d.shape[0]}, {self.w.d.shape[1]})'

In [ ]:
#export

class ReLU(Module):
    def forward(self, x): return x.clamp_min_(0.)-0.5
    def bwd(self, out, inp):
        inp.g = (inp>0).float() * out.g
    def __repr__(self): return f'ReLU()'

In [ ]:
model = SequentialModel(Linear(n_in,50, True), ReLU(), Linear(50,n_out, False))

In [ ]:
model

(Layer1): Linear(784, 50)
(Layer2): ReLU()
(Layer3): Linear(50, 10)

In [ ]:
for p in model.parameters(): print(p)

Parameter: torch.Size([784, 50]), Requires grad: True
Parameter: torch.Size([50]), Requires grad: True
Parameter: torch.Size([50, 10]), Requires grad: True
Parameter: torch.Size([10]), Requires grad: True


## Optimizer

In [ ]:
#export
class Optimizer():
    def __init__(self, params, lr): self.params, self.lr = list(params), lr
        
    def step(self): 
        for p in self.params: p.step(self.lr)
    
    def zero_grad(self): 
        for p in self.params: p.zero_grad()

In [ ]:
optim = Optimizer(model.parameters(), 0.5)

In [ ]:
def fit(epochs, model, optim, loss_func, train, valid):
    "Fit function 2: Added easier optimization steps"
    for epoch in range(epochs):
        for batch in range(math.ceil(len(train)//bs)):
            start = batch*bs
            end = batch*bs + bs
            train_batch = train[start:end]
            valid_batch = valid[start:end]
            
            loss = loss_func(model(train_batch), valid_batch)
            loss_func.backward()
            model.backward()
            
            optim.step()
            optim.zero_grad()
            
        print(f'Epoch {epoch+1}, Accuracy: {accuracy(model(xt), yt)}')

In [ ]:
fit(3, model, optim, CrossEntropy(), xt, yt)

Epoch 1, Accuracy: 0.8959000110626221
Epoch 2, Accuracy: 0.9216799736022949
Epoch 3, Accuracy: 0.9330599904060364


In [ ]:
#export
def get_model(lr):
    "Easy helper function to get basic fully connected network with optimizer and loss function, takes learning rate, `lr`, as a parameter"
    model = SequentialModel(Linear(784, 50, True), ReLU(), Linear(50, 10, False)) 
    loss_func = CrossEntropy()
    optimizer = Optimizer(model.parameters(), lr)
    return model, optimizer, loss_func

## Databunch

In [ ]:
#export
class Dataset():
    "Container class to store and get input and target values from a dataset"
    def __init__(self, x, y): self.x, self.y = x, y
        
    def __getitem__(self, i): return self.x[i], self.y[i]
    
    def __len__(self): return len(self.x)

In [ ]:
class DataLoader():
    "Container class to iterate over a dataset given a batcht size"
    def __init__(self, ds, bs): 
        "Provide a dataset, `ds`, and batchsize, `bs`"
        self.ds, self.bs = ds,bs
    def __iter__(self): 
        for i in range(0, len(self.ds), self.bs): yield self.ds[i:i+self.bs]

In [ ]:
ds = Dataset(xt, yt)
dl = DataLoader(ds, bs)

In [ ]:
def fit(epochs, model, optim, loss_func, data_loader):
    "Training Loop 3: Refactored out for easy databunch usage"
    for epoch in range(epochs):
        for xb, yb in data_loader:
            loss = loss_func(model(xb), yb)
            loss_func.backward()
            model.backward()
            
            optim.step()
            optim.zero_grad()
            
        print(f'Epoch {epoch+1}, Accuracy: {accuracy(model(xt), yt)}')

In [ ]:
m, o, lf = get_model(0.5)
fit(3, m, o, lf, dl)

Epoch 1, Accuracy: 0.9178799986839294
Epoch 2, Accuracy: 0.9384199976921082
Epoch 3, Accuracy: 0.9474200010299683


## Random Data

In [ ]:
#export
class Batcher():
    "Wrapper for databunch class that randomizes each batch of output if `random` arg is set to trueu"
    def __init__(self, ds, bs, random): 
        self.n, self.bs, self.rand = len(ds), bs, random
    def __iter__(self):
        "When iter is called, random batches of the dataset are created"
        self.idxs = torch.randperm(self.n) if self.rand else torch.arange(self.n)
        for i in range(0, self.n, self.bs): yield self.idxs[i:i+self.bs]

In [ ]:
small_ds = Dataset(*ds[:10])

In [ ]:
random = Batcher(small_ds, 4, True)
not_random = Batcher(small_ds, 4, False)

In [ ]:
[r for r in random]

[tensor([6, 5, 1, 7]), tensor([4, 3, 8, 2]), tensor([0, 9])]

In [ ]:
[nr for nr in not_random]

[tensor([0, 1, 2, 3]), tensor([4, 5, 6, 7]), tensor([8, 9])]

In [ ]:
#export
def collate(b):
    "Combines the input lists `b` into single inpuut and target tensors"
    xb, yb = zip(*b)
    return torch.stack(xb), torch.stack(yb)
    
    
class DataLoader():
    "Refactored DataLoader to include a batcher, also collates the output of batcher into single tensor for model"
    def __init__(self, ds, batcher, collate_fcn): self.ds, self.batcher, self.collate_fcn = ds, batcher, collate_fcn
    def __iter__(self): 
        for b in self.batcher: yield self.collate_fcn([self.ds[i] for i in b])
    def __len__(self): return len(self.ds)

## Validation

In [ ]:
#export
def fit(epochs, model, optim, loss_func, train, valid):
    "Fit function 4: Added validation loops, model training status as well as printing of some metrics"
    for epoch in range(epochs):
        
        model.training = True
        for xb, yb in train:
            loss = loss_func(model(xb), yb)
            loss_func.backward()
            model.backward()
            
            optim.step()
            optim.zero_grad()
        
        model.training = False
        acc, loss, epochs = 0,0,0
        for xb, yb in valid:
            pred = model(xb)
            acc += accuracy(pred, yb)
            loss += loss_func(pred, yb)
            epochs += 1
        acc /= epochs
        loss /= epochs
            
        print(f'Epoch {epoch+1}, Accuracy: {acc}, Loss: {loss}')

In [ ]:
#export
def get_datasets():
    "Helper function to return proper dataloaders"
    xt, yt, xv, yv = get_mnist()
    tr = Dataset(xt, yt)
    val = Dataset(xv, yv)
    train = DataLoader(tr, Batcher(tr, 64, True), collate)
    valid = DataLoader(val, Batcher(val, 64, False), collate)
    return train, valid

In [ ]:
train, valid = get_datasets()
m, o, lf = get_model(0.1)
fit(3, m, o, lf, train, valid)

Epoch 1, Accuracy: 0.9465565085411072, Loss: 0.18650124967098236
Epoch 2, Accuracy: 0.9217754602432251, Loss: 0.24221312999725342
Epoch 3, Accuracy: 0.9668591022491455, Loss: 0.11605237424373627
